In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import gc
import collections

<h2> Custom Gradient Descent </h2>

In [7]:
intmatrix = np.load("./data/movie-matrix.npz")["data"]
#Let us define 512-length vector embeddings for users and items such that we prefer higher inner products
latentdim = 512
Usr = np.random.normal(loc=0, scale=1, size=(intmatrix.shape[0], latentdim))
Itm = np.random.normal(loc=0, scale=1, size=(latentdim, intmatrix.shape[1]))

In [10]:
mean = np.sum(intmatrix, axis=1)/(intmatrix!=0).sum(axis=1)
zmask = intmatrix==0
intmatrix = intmatrix-mean.reshape(-1,1)
intmatrix[zmask]=0

In [19]:
#Define Loss
#We add the inner products of all users-items above user-mean and subtract the inner products of all user-items
#below mean. We must maximize this quantity, and hence minimize its negation
def embed_loss(Usr, Itm, intmatrix, wbl=0):
    mat = np.matmul(Usr, Itm)
    posquant = mat[intmatrix>0].sum()
    negquant = mat[intmatrix<0].sum()
    return negquant-posquant

In [20]:
embed_loss(Usr, Itm, intmatrix)

-6662.141534947114